# Study of a Regression problem on Franke function

In [1]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from random import seed
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
import warnings
warnings.filterwarnings("ignore")

from functions import *

from activation_funcs import *
from cost_funcs import *
from Schedulers import *
from FFNN import FFNN

seed = 42069
np.random.seed(seed)

In [2]:
N  = 40
x = np.arange(0, 1, 1/N)
y = np.arange(0, 1, 1/N)
X, Y = np.meshgrid(x, y)

z = (FrankeFunction(X, Y) + np.random.normal(0, 0.1, (N, N))).reshape(-1, 1) 

poly_degree = 3
X = create_X(x, y, poly_degree)

X_train, X_test, y_train, y_test = train_test_split(X, z, test_size=0.2, random_state=seed)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X_train_scaled.shape)

(1280, 10)


In [3]:
input_shape = X_train_scaled.shape[1]
hidden_shape = (2, 2)
output_shape = 1
dims =  (input_shape, *hidden_shape, output_shape)

model = FFNN(
    dimensions=dims,
    hidden_funcs=(sigmoid, sigmoid),
    output_func=identity,
    cost_func=MSE,
    cost_der=MSE_derivative,
    seed=seed
)

In [4]:
scheduler = Adagrad(eta=0.01)

epochs = 1000
batch_size = 10

lmbda = 0.01

scores = model.fit(
    X_train_scaled,
    y_train, 
    scheduler=scheduler,
    epochs=epochs,
    batch_size=batch_size,
    lmbda=lmbda,
    )

2


IndexError: list index out of range